In [1]:
import os

while "src" not in os.listdir():
    os.chdir("..")
    print(f"Current folder: {os.getcwd()}")

Current folder: c:\Users\giova\Desktop\MLCube\Repositories\ml3\rag-with-polars


# Imports

In [2]:
import polars as pl
import numpy as np
from dotenv import load_dotenv
import requests

In [3]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
url = "https://api.openai.com/v1/embeddings"

In [4]:
def compute_embeddings(
    texts: str | list[str],
    open_ai_model: str = "text-embedding-3-small",
    normalize: bool = True,
) -> np.ndarray:
    """
    Compute embeddings for a list of texts.

    Returns a numpy array of embeddings of shape
    (len(texts), embedding_size).
    """

    if isinstance(texts, str):
        texts = [texts]

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "input": texts,
        "model": open_ai_model,
        "encoding_format": "float",
    }

    # Make the POST request
    json_response = requests.post(url, headers=headers, json=payload).json()

    # Extract the embeddings

    embeddings = np.array(
        [embedding_json["embedding"] for embedding_json in json_response["data"]]
    )

    if normalize:
        embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    return embeddings

# Load Data

In [5]:
DATA_FOLDER = "data"
DATASET_NAME = "hacker_news"
FILE_NAME = f"{DATA_FOLDER}/{DATASET_NAME}.parquet"

In [6]:
df = pl.scan_parquet(FILE_NAME)

In [7]:
materialized_df = df.collect()

In [8]:
materialized_df.shape

(28544, 5)

In [9]:
df.head().collect()

id,time,title,url,embedding
i32,i32,str,str,"array[f64, 1536]"
35515614,1681151391,"""Text-Based Tetris""","""https://aino.agency/game""","[-0.041159, 0.038379, … 0.001997]"
35680911,1682285922,"""Will the Internet Democratize …","""https://www.nytimes.com/2023/0…","[0.020964, -0.022481, … -0.008165]"
35806111,1683139428,"""ChatGPT can now find you a hou…","""https://www.theverge.com/2023/…","[-0.03301, 0.025399, … -0.000259]"
35908618,1683840510,"""Capsule captures first look in…","""https://www.ucdavis.edu/news/c…","[-0.004219, 0.024209, … -0.011713]"
35911041,1683857335,"""Long popular in Asia, floating…","""https://apnews.com/article/flo…","[-0.0095, 0.006706, … -0.000616]"


df is our vector store. It has an id, the title, the url of the post, the time it was published and the embedding

In [10]:
query = "Python"
query_embedding = compute_embeddings(query)

### Method 1

Inspired by the blog post. Materialize dataframe, get embedding column, and convert to numpy array and compute cosine similarity with efficient
numpy tricks.

In [11]:
def get_closest_1(
    lazy_vector_store: pl.LazyFrame,
    query_embed: np.ndarray,
    materialized_vector_store: pl.DataFrame | None = None,
    k: int = 3,
) -> pl.DataFrame:
    """
    First method: extract embeddings column and compute
    cosine similarity with the query embedding.

    Returns the k rows with the smallest cosine similarity.

    The performance of this method is heavily impacted by having
    to materialize the dataframe. We keep that as an argument
    so that we can test its impact.
    Nonetheless assuming to having it already materialized is not
    optimal since in a real-world scenario the embeddings could be
    very large (thus not fitting in memory) or we might want to load
    them "on-demand" (rather than having them always in memory).
    """

    # We need to materialize in order to access the embeddings column
    if materialized_vector_store is None:
        materialized_vector_store = lazy_vector_store.select(["embedding"]).collect()

    vector_store_embeds = materialized_vector_store["embedding"].to_numpy()

    # Compute cosine similarity.
    # Since the embeddings are normalized, this is equivalent to the dot product.
    cosine_similarities = np.einsum("ij,ij->i", vector_store_embeds, query_embed)

    # Get the indices of the k smallest cosine similarities
    # Notice that argpartition gives no guarantee on the order
    # of the k smallest elements, which is why we need
    # an extra sorting step after the partitioning.
    closest_indices = np.argpartition(cosine_similarities, -k)[-k:]

    # Sort the k closest indices by cosine similarity
    idx = closest_indices[np.argsort(cosine_similarities[closest_indices])[::-1]]

    return (
        lazy_vector_store.with_row_index()
        .filter(pl.col("index").is_in(idx))
        .select(["id", "title", "url", "time"])
        .collect()
    )

In [12]:
get_closest_1(lazy_vector_store=df, query_embed=query_embedding, k=3)

id,title,url,time
i32,str,str,i32
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151


In [13]:
get_closest_1(
    lazy_vector_store=df,   
    query_embed=query_embedding,
    k=3,
    materialized_vector_store=materialized_df,
)

id,title,url,time
i32,str,str,i32
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151


In [14]:
%%timeit

get_closest_1(lazy_vector_store=df, query_embed=query_embedding, k=3)

1.08 s ± 66.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

get_closest_1(
    lazy_vector_store=df,
    query_embed=query_embedding,
    k=3,
    materialized_vector_store=materialized_df,
)

36.1 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Method 2



In [16]:
def get_closest_2(
    lazy_vector_store: pl.LazyFrame,
    query_embed: np.ndarray,  # noqa
    materialized_vector_store: pl.DataFrame | None = None,  # noqa
    k: int = 3,
) -> pl.DataFrame:
    """ """

    return (
        lazy_vector_store.with_columns(
            query=pl.lit(query_embed.reshape(-1).tolist()).cast(
                pl.Array(pl.Float64, shape=query_embed.shape[1])
            ),
        )
        .with_columns(
            sim=np.dot(
                pl.col("embedding"),
                pl.col("query"),
            ).arr.sum()
        )
        .top_k(k, by="sim")
        .select(["id", "title", "url", "time"])
        .collect()
    )

In [17]:
# def get_closest_2_v2(
#     lazy_vector_store: pl.LazyFrame,
#     query_embed: np.ndarray,  # noqa
#     materialized_vector_store: pl.DataFrame | None = None,  # noqa
#     k: int = 3,
# ) -> pl.DataFrame:
#     """ """

#     return (
#         lazy_vector_store.with_columns(
#             query=pl.lit(query_embed.reshape(-1).tolist()).cast(
#                 pl.Array(pl.Float64, shape=query_embed.shape[1])
#             ),
#         )
#         .with_columns(
#             dot=np.dot(
#                 pl.col("embedding"),
#                 pl.col("query"),
#             )
#         )
#         .with_columns(
#             sim=pl.col("dot").arr.sum()
#         )
#         .top_k(k, by="sim")
#         .select(["id", "title", "url", "time"])
#         .collect()
#     )

In [18]:
get_closest_2(
    lazy_vector_store=df,
    query_embed=query_embedding,
    k=3,
)

id,title,url,time
i32,str,str,i32
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178


In [19]:
%%timeit

get_closest_2(lazy_vector_store=df, query_embed=query_embedding, k=3)

1.28 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Method 3

Sort by rank.

In [20]:
def get_closest_3(
    lazy_vector_store: pl.LazyFrame,
    query_embed: np.ndarray,  # noqa
    materialized_vector_store: pl.DataFrame | None = None,  # noqa
    k: int = 3,
) -> pl.DataFrame:
    """ """

    return (
        lazy_vector_store.with_columns(
            query=pl.lit(query_embed.reshape(-1).tolist()).cast(
                pl.Array(pl.Float64, shape=query_embed.shape[1])
            ),
        )
        .with_columns(
            rank=np.dot(
                pl.col("embedding"),
                pl.col("query"),
            )
            .arr.sum()
            .rank(method="min")
        )
        .top_k(k, by="rank")
        .select(["id", "title", "url", "time"])
        .collect()
    )

In [21]:
get_closest_3(
    lazy_vector_store=df,
    query_embed=query_embedding,
    k=3,
)

id,title,url,time
i32,str,str,i32
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178


In [22]:
%%timeit

get_closest_3(lazy_vector_store=df, query_embed=query_embedding, k=3)

1.18 s ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Method 4

Compute rank, then materialize dataframe, sort by rank in numpy, get top k.

In [23]:
def get_closest_4(
    lazy_vector_store: pl.LazyFrame,
    query_embed: np.ndarray,  # noqa
    materialized_vector_store: pl.DataFrame | None = None,  # noqa
    k: int = 3,
) -> pl.DataFrame:
    """ """

    df_with_query = lazy_vector_store.with_columns(
        query=pl.lit(query_embed.reshape(-1).tolist()).cast(
            pl.Array(pl.Float64, shape=query_embed.shape[1])
        ),
    ).with_columns(
        rank=np.dot(
            pl.col("embedding"),
            pl.col("query"),
        )
        .arr.sum()
        .rank(method="min")
    )

    # sort in numpy
    materialized_rank = df_with_query.select(["rank"]).collect()["rank"].to_numpy()
    closest_indices = np.argpartition(materialized_rank, -k)[-k:]

    # Sort the k closest indices by cosine similarity
    idx = closest_indices[np.argsort(materialized_rank[closest_indices])[::-1]]

    return (
        df_with_query.with_row_index()
        .filter(pl.col("index").is_in(idx))
        .select(["id", "title", "url", "time"])
        .collect()
    )

In [24]:
get_closest_4(
    lazy_vector_store=df,
    query_embed=query_embedding,
    k=3,
)

id,title,url,time
i32,str,str,i32
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151


In [25]:
%%timeit

get_closest_4(lazy_vector_store=df, query_embed=query_embedding, k=3)

1.21 s ± 61.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Method 5

Compute rank and get top k by filtering (no sorting)

In [26]:
def get_closest_5(
    lazy_vector_store: pl.LazyFrame,
    query_embed: np.ndarray,  # noqa
    materialized_vector_store: pl.DataFrame | None = None,  # noqa
    k: int = 3,
) -> pl.DataFrame:
    """ """

    return (
        lazy_vector_store.with_columns(
            query=pl.lit(query_embed.reshape(-1).tolist()).cast(
                pl.Array(pl.Float64, shape=query_embed.shape[1])
            ),
        )
        .with_columns(
            rank=np.dot(
                pl.col("embedding"),
                pl.col("query"),
            )
            .arr.sum()
            .neg()
            .rank(method="min")
        )
        .filter(pl.col("rank") <= k)
        .select(["id", "title", "url", "time"])
        .collect()
    )


In [27]:
get_closest_5(
    lazy_vector_store=df,
    query_embed=query_embedding,
    k=3,
)

id,title,url,time
i32,str,str,i32
35421096,"""Think Python 2e""","""https://greenteapress.com/wp/t…",1680517178
35465484,"""Python Programming Exercises G…","""https://inventwithpython.com/p…",1680767014
35820038,"""Python How Tos""","""https://docs.python.org/3/howt…",1683225151


In [28]:
%%timeit

get_closest_5(lazy_vector_store=df, query_embed=query_embedding, k=3)

1.19 s ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
